## This example jupyer notebook uses clmm code to generate mock data with a set of configuration parameters, producing a catalog of ideal and noisy example data.

In [ ]:
import numpy as np
import pyccl as ccl
import cluster_toolkit as ct
import matplotlib.pyplot as plt
%matplotlib inline

### Import mock data module and setup the configuration 

In [ ]:
import sys
sys.path.append('./support')
sys.path.append('../clmm')
import modeling as pp
import mock_data as mock


### Mock data generation requires a config dictionary

In [ ]:
config={}
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4

config['cosmo'] = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96)

# If CCL is not installed use instead of the above line
# from astropy.cosmology import FlatLambdaCDM
# astropy_cosmology_object = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)
# config['cosmo'] = pp.create_ccl_cosmo_object_from_astropy(astropy_cosmology_object)

config['ngals'] = 10000
config['Delta'] = 200

ideal_data = mock.MockData(config=config)
noisy_data = mock.MockData(config=config)
noisy_data_z = mock.MockData(config=config)

### Generate the mock catalog from the configuration. Consider 3 examples of increasing complexity:
* Ideal data, no noise, all galaxies at the same redshift
* Noisy data, including shape noise and redshift error, all galaxies at the same redshift
* Noisy data, galaxies following redshift distribution, redshift error, shape noise

In [ ]:
ideal_data.generate()
noisy_data.generate(is_shapenoise=True, shapenoise=0.005, is_zerr=True)
noisy_data_z.generate(is_shapenoise=True, is_zdistribution=True, is_zerr=True)


#### Ideal catalog first entries: no noise on the shape measurement, all galaxies at z=0.3

In [ ]:
ideal_data.catalog[0:3]

#### More realistic catalog: noise has been added on the shape measurement, galaxies follow some photoz distribution, gaussian pdf for each photoz is also provided (along with corresponding bins)

In [ ]:
noisy_data.catalog[0:3]

In [ ]:
noisy_data_z.catalog[0:3]

In [ ]:
# Histogram of the redshift distribution of bkg galaxies (starting at z_cluster + 0.1)
hist = plt.hist(noisy_data_z.catalog['z'], bins=50)

In [ ]:
# pdz for the first galaxy in the catalog
plt.plot(noisy_data_z.catalog['z_bins'][0],noisy_data_z.catalog['z_pdf'][0])

### Compute and plot shear profile

In [ ]:
# from clmm import ShearAzimuthalAverager

In [ ]:
# cl_dict = {'z':config['cluster_z'], 'ra':0.0, 'dec': 0.0}

# saa_ideal = ShearAzimuthalAverager(cl_dict,ideal_data.catalog)
# saa_noisy = ShearAzimuthalAverager(cl_dict,noisy_data.catalog)
# saa_noisy_z = ShearAzimuthalAverager(cl_dict,noisy_data_z.catalog)


# saa_ideal.compute_shear()
# saa_noisy.compute_shear()
# saa_noisy_z.compute_shear()

# saa_ideal.make_shear_profile()
# saa_noisy.make_shear_profile()
# saa_noisy_z.make_shear_profile()

In [ ]:
# Ideal data
# saa_ideal.plot_profile()

In [ ]:
# Noisy data, all galaxies at the same redshift
# saa_noisy.plot_profile()

In [ ]:
# Noisy data, galaxies following redshift distribution
# saa_noisy_z.plot_profile()